In [60]:
from langchain.vectorstores import Weaviate
from dotenv import load_dotenv
import os
import weaviate
from sentence_transformers import SentenceTransformer

In [61]:
load_dotenv()
WEAVIATE_API_KEY=os.getenv('weaviateapikey')
WEAVIATE_CLUSTER=os.getenv('WEAVIATECLUSTER')
huggingfacehub_api_token=os.getenv('huggingfacehubapitoken')

In [62]:
WEAVIATE_CLUSTER

'https://mylangchainproject-ilhnm5qv.weaviate.network'

In [63]:
client = weaviate.Client(
    url = WEAVIATE_CLUSTER,
    auth_client_secret=weaviate.auth.AuthApiKey(api_key=WEAVIATE_API_KEY),  # Replace with your Weaviate instance API key
)

a:\LANGCHAIN\langchainenv\lib\site-packages\weaviate\warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(


In [64]:
## specify embedding model using huggingface
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model_name='sentence-transformers/all-mpnet-base-v2'
embeddings=HuggingFaceEmbeddings(
    model_name=embedding_model_name
)

In [65]:
from langchain.document_loaders import PyPDFLoader
loader=PyPDFLoader("Data_source/Retrieval-Augmented Generation for Knowledge intensive NLP tasks.pdf",extract_images=True)
pages=loader.load()

Windows platform detected, try to use DirectML as primary provider
Windows platform detected, try to use DirectML as primary provider
Windows platform detected, try to use DirectML as primary provider
Windows platform detected, try to use DirectML as primary provider
Windows platform detected, try to use DirectML as primary provider
Windows platform detected, try to use DirectML as primary provider
Windows platform detected, try to use DirectML as primary provider
Windows platform detected, try to use DirectML as primary provider
Windows platform detected, try to use DirectML as primary provider
Windows platform detected, try to use DirectML as primary provider
Windows platform detected, try to use DirectML as primary provider
Windows platform detected, try to use DirectML as primary provider


In [66]:
## split text into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=20)
docs=text_splitter.split_documents(pages)

In [67]:
vector_db=Weaviate.from_documents(
    docs,embeddings,client=client,by_text=False
)

In [73]:
print(vector_db.similarity_search("What is rag?",k=3)[0].page_content)

documents for questions that are less likely to beneﬁt from retrieval, suggesting that null document
mechanisms may not be necessary for RAG.
G Parameters
Our RAG models contain the trainable parameters for the BERT-base query and document encoder of
DPR, with 110M parameters each (although we do not train the document encoder ourselves) and
406M trainable parameters from BART-large, 406M parameters, making a total of 626M trainable
18


In [74]:
from langchain.prompts import ChatPromptTemplate
template="""You are an assistant for question-ansering tasks.
Use the following pieces of retrieved context to answer the quetsion.
If you don't know the answer , just say that you don't know
USe twenty sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Anwser:
"""

In [75]:
prompt=ChatPromptTemplate.from_template(template=template)

In [76]:
from langchain import HuggingFaceHub

In [77]:
model=HuggingFaceHub(
    huggingfacehub_api_token=huggingfacehub_api_token,
    repo_id="mistralai/Mistral-7B-Instruct-v0.2"

)

In [78]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [79]:
output_parser=StrOutputParser()
retriever=vector_db.as_retriever()

In [80]:
rag_chain=({
    "context":retriever,"question":RunnablePassthrough()}
    | prompt
    | model
    | output_parser
    )

In [85]:
print(rag_chain.invoke("What is Rag System?"))

Human: You are an assistant for question-ansering tasks.
Use the following pieces of retrieved context to answer the quetsion.
If you don't know the answer , just say that you don't know
USe twenty sentences maximum and keep the answer concise.
Question: What is Rag System?
Context: [Document(page_content='Table 2 shows our results on FEVER. For 3-way classiﬁcation, RAG scores are within 4.3% of\nstate-of-the-art models, which are complex pipeline systems with domain-speciﬁc architectures and\nsubstantial engineering, trained using intermediate retrieval supervision, which RAG does not require.\n6', metadata={'page': 5, 'source': 'Data_source/Retrieval-Augmented Generation for Knowledge intensive NLP tasks.pdf'}), Document(page_content='documents for questions that are less likely to beneﬁt from retrieval, suggesting that null document\nmechanisms may not be necessary for RAG.\nG Parameters\nOur RAG models contain the trainable parameters for the BERT-base query and document encoder of

In [87]:
rag_chain.invoke("What is Rag System?")['answer']

TypeError: string indices must be integers